In [1]:
import pandas as pd
import os

In [2]:
import sys  
sys.path.insert(0, 'src')
from model import MCSE, BertForCL, RobertaForCL, ResNetVisnModel

In [3]:
sys.path.insert(0, 'preprocess')
from extract_visn_feature import ResnetFeatureExtractor

In [4]:
refann_df = pd.read_csv('data/ref_data_union.csv', index_col=0)
display(refann_df)

,article,text,sentence,start,end,local_ix,par_ix,ax,related_images,ref_no,tokens,aat_terms,all_terms,term_density,term_density_gen
43,Exeter Cathedral,"Exeter Cathedral, properly known as the Cathed...","Exeter Cathedral, properly known as the Cathed...",0,199,0,8,43,"['File:Exeter_Cathedral_Nave,_Exeter,_UK_-_Dil...",3,"['exeter', 'cathedral', ',', 'properly', 'know...","['cathedral', 'cathedral', 'church', 'cathedra...","['cathedral', 'cathedral', 'church', 'cathedra...",0.166667,0.166667
44,Exeter Cathedral,"Exeter Cathedral, properly known as the Cathed...",The present building was complete by about 140...,200,613,0,8,44,"['File:Exeter_Cathedral_Nave,_Exeter,_UK_-_Dil...",5,"['the', 'present', 'building', 'wa', 'complete...","['building', 'set', 'misericords', 'clock', 's...","['building', 'set', 'misericords', 'clock', 's...",0.136986,0.136986
45,Exeter Cathedral,"Exeter Cathedral, properly known as the Cathed...",A Saxon minster already existing within the to...,615,838,0,8,45,['File:Exeter_Cathedral_9578.jpg'],1,"['a', 'saxon', 'minster', 'already', 'existing...","['minster', 'town', 'seat', 'service', 'door',...","['minster', 'town', 'seat', 'service', 'door',...",0.200000,0.200000
46,Exeter Cathedral,In 1107 William Warelwast was appointed to the...,In 1107 William Warelwast was appointed to the...,0,134,1,9,46,[],0,"['in', '1107', 'william', 'warelwast', 'wa', '...","['catalyst', 'building', 'cathedral']","['catalyst', 'building', 'cathedral']",0.111111,0.111111
47,Exeter Cathedral,In 1107 William Warelwast was appointed to the...,"Its official foundation was in 1133, during Wa...",136,238,1,9,47,[],0,"['it', 'official', 'foundation', 'wa', 'in', '...",['foundation'],['foundation'],0.047619,0.047619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10433,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,This was due to a monetary disagreement that l...,866,1050,2,1899,10433,[],0,"['this', 'wa', 'due', 'to', 'a', 'monetary', '...","['led', 'statue', 'home']","['led', 'statue', 'home']",0.090909,0.090909
10434,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,The sculpture was a memorial to the Mouscron p...,1051,1329,2,1899,10434,[],0,"['the', 'sculpture', 'wa', 'a', 'memorial', 't...","['sculpture', 'memorial', 'tabernacle', 'sculp...","['sculpture', 'memorial', 'parent', 'tabernacl...",0.113208,0.132075
10435,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,"While Michelangelo was alive, the Madonna was ...",1330,1420,2,1899,10435,[],0,"['while', 'michelangelo', 'wa', 'alive', ',', ...",['sculpture'],['sculpture'],0.055556,0.055556
10436,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,The sculpture was twice recovered after being ...,1421,1551,2,1899,10436,[],0,"['the', 'sculpture', 'wa', 'twice', 'recovered...",['sculpture'],['sculpture'],0.050000,0.050000


In [5]:
pic_df = pd.read_csv('data/image_data.csv')

In [6]:
display(pic_df)

,Unnamed: 0,article,wiki_url,caption,filename
0,3,Exeter Cathedral,https://upload.wikimedia.org/wikipedia/commons...,The cathedral in 1830,Exeter_Cathedral_NW_view_W_Deeble_after_R_Brow...
1,4,Exeter Cathedral,https://upload.wikimedia.org/wikipedia/commons...,"Inside the cathedral, showing the vaulted ceil...",Inside_Exeter_Cathedral.jpg
2,5,Exeter Cathedral,https://upload.wikimedia.org/wikipedia/commons...,Detail of the vaulted ceiling,Exeter_Cathedral_nave_vaulted_ceiling.jpg
3,6,Exeter Cathedral,https://upload.wikimedia.org/wikipedia/commons...,"One of the misericords, depicting a pipe and t...",Misericordtaborer.png
4,7,Exeter Cathedral,https://upload.wikimedia.org/wikipedia/commons...,The Minstrels' Gallery,Exeter_Cathedral_Minstrels'_Gallery.jpg
...,...,...,...,...,...
1002,1930,"Church of Our Lady, Bruges",https://upload.wikimedia.org/wikipedia/commons...,"Madonna by Michelangelo, 1501-1504",Michelangelo's_Madonna_and_Child_in_Brugge.jpg
1003,1931,"Church of Our Lady, Bruges",https://upload.wikimedia.org/wikipedia/commons...,Madonna by Michelangelo,Michelangelos_Madonna_OLV-Kerk_Brugge.jpg
1004,1932,"Church of Our Lady, Bruges",https://upload.wikimedia.org/wikipedia/commons...,"Crucifixion by Anthony van Dyck, 1626",Antoon_van_Dijck.Kruisiging.JPG
1005,1933,"Church of Our Lady, Bruges",https://upload.wikimedia.org/wikipedia/commons...,"Supper at Emmaüs, formerly ascribed to Caravaggio",Caravaggio_Liebfrauenkirche_Abendmahl_in_Emmau...


In [29]:
paths = [p for p in list(pic_df["filename"]) if p in os.listdir("medium_images")]

In [8]:
len(paths)

994

In [22]:
extractor = ResnetFeatureExtractor("medium_images", "data/", 32)
extractor.extract_vision_features(dataname="medium_images", img_ids=paths)

  2%|█▋                                                                               | 20/994 [00:00<00:10, 96.72it/s]

Start extracting resnet features...


100%|████████████████████████████████████████████████████████████████████████████████| 994/994 [00:36<00:00, 27.51it/s]

	Save features to data/medium_images.hdf5 with hdf5 dataset 'features'.


## Sentences 

In [37]:
from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data.dataloader import DataLoader
import numpy as np
import tqdm
import h5py

In [10]:
model_path = "result/mix_flickr/0/mcse"

In [11]:
model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
device = torch.device("cuda:0")
model = model.to(device)

In [12]:
sentences = list(refann_df["sentence"])

In [33]:
def extract_sent_features(sents, model, batch_size, cls=False):
    tensor_sents = []
    sent_feats = []
    for i, sent in enumerate(tqdm.tqdm(sents)):
        tensor_sents.append(sent)
        if len(tensor_sents) == batch_size:
            tensor_sents = tokenizer.batch_encode_plus(tensor_sents,return_tensors='pt',padding=True,max_length=32,
                                                       truncation=True)
            for k in tensor_sents:
                tensor_sents[k] = tensor_sents[k].to(device)
            #sent_input = torch.stack(tensor_sents).cuda()
            #sent_input = torch.squeeze(sent_input)
            with torch.no_grad():
                sent_output = model(**tensor_sents)
            sent_feats.extend(sent_output.last_hidden_state.detach().cpu().numpy())
            tensor_sents = []
    if len(tensor_sents) > 0: 
        tensor_sents = tokenizer.batch_encode_plus(tensor_sents,return_tensors='pt',padding=True,max_length=32,
                                                   truncation=True)
        for k in tensor_sents:
            tensor_sents[k] = tensor_sents[k].to(device)
        #sent_input = torch.stack(tensor_sents).cuda()
        #sent_input = torch.squeeze(sent_input)
        with torch.no_grad():
            sent_output = model(**tensor_sents)
        sent_feats.extend(sent_output.last_hidden_state.detach().cpu().numpy())
        sent_feats = np.stack(sent_feats)
        if cls: 
            sent_feats = sent_feats[:, 0]
    return(sent_feats)

                
    

In [34]:
def save_to_hdf5(feats, output_file):
    h5f = h5py.File(output_file, 'w')
    dset = h5f.create_dataset("features", data=feats)
    h5f.close()
    print("features saved to " + output_file)

In [35]:
#sent_feats = extract_sent_features(sentences, model, 32)
cls_feats = extract_sent_features(sentences, model, 32, cls=True)

100%|█████████████████████████████████████████████████████████████████████████████| 5066/5066 [00:50<00:00, 100.29it/s]


In [38]:
save_to_hdf5(cls_feats, "data/cls_feats.hdf5")

features saved to data/cls_feats.hdf5


In [39]:
### opening the .hdf5 files

In [48]:
hf = h5py.File('data/sent_feats.hdf5', 'r')
#data = hf.get('features').value
data = np.array(hf["features"][:])

data.shape

(5066, 32, 768)

In [40]:
cls_feats.shape

(5066, 768)

## Sonstiges

In [17]:
from PIL import Image

for p in listdir("medium_images"):
    if p.endswith(".png"):
        fp = "medium_images/" + p
        im = Image.open(fp)
        rgb_im = im.convert('RGB')
        rgb_im.save(os.path.splitext(fp)[0] + ".jpg")

In [15]:
import os

conts = [p.lower() for p in os.listdir("medium_images")]
missing = []
for p in paths: 
    if not p.lower() in conts:
        missing.append(p)

In [16]:
print(len(missing))

13
